Install packages

In [2]:
pip install pandas tensorflow matplotlib numpy tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 727.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 136.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00


Import packages

In [3]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaymann/handwritten-english-characters-and-digits")

print("Path to dataset files:", path)

100%|██████████| 205M/205M [00:01<00:00, 193MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sujaymann/handwritten-english-characters-and-digits/versions/6


In [5]:
import os
print("Files in dataset folder:")
print(os.listdir(path))

Files in dataset folder:
['augmented_images', 'image_labels.csv', 'handwritten-english-characters-and-digits']


In [6]:
train_dir = os.path.join(path, "handwritten-english-characters-and-digits/combined_folder/train") #'/kaggle/input/handwritten-english-characters-and-digits/handwritten-english-characters-and-digits/combined_folder/train'
test_dir  = os.path.join(path, "handwritten-english-characters-and-digits/combined_folder/test") #'/kaggle/input/handwritten-english-characters-and-digits/handwritten-english-characters-and-digits/combined_folder/test'
augmented_data= os.path.join(path, "augmented_images/augmented_images1") #'/kaggle/input/handwritten-english-characters-and-digits/augmented_images/augmented_images1'

Data preprocessing

In [7]:
validate_ds=tf.keras.utils.image_dataset_from_directory(
    train_dir, # root directory containing per-class subfolders
    image_size=(32,32),  # resize every image to 32x32 (bilinear interpolation)
    batch_size=32, # number of samples per batch
    label_mode='categorical' # return one-hot labels (shape: [B, num_classes])
)

Found 2728 files belonging to 62 classes.


In [8]:
augmented_ds=tf.keras.utils.image_dataset_from_directory(
    augmented_data, # root directory containing per-class subfolders
    image_size=(32,32), # resize every image to 32x32 (bilinear interpolation)
    batch_size=32, # number of samples per batch
    label_mode='categorical'# return one-hot labels (shape: [B, num_classes])
)

Found 13640 files belonging to 62 classes.


In [9]:
test_ds=tf.keras.utils.image_dataset_from_directory(
    test_dir, # root directory containing per-class subfolders
    image_size=(32,32), # resize every image to 32x32 (bilinear interpolation)
    batch_size=32, # number of samples per batch
    label_mode='categorical'# return one-hot labels (shape: [B, num_classes])
)

Found 682 files belonging to 62 classes.


In [10]:

# Function to convert dataset to flattened NumPy arrays
def dataset_to_numpy(ds):
    ds = ds.unbatch()
    X = []
    y = []
    for image, label in tqdm(ds):
        X.append(tf.reshape(image, [-1]).numpy())  # Flatten image
        y.append(tf.argmax(label).numpy())         # Convert one-hot to label index
    return np.array(X), np.array(y)

# Convert training data
X_train, y_train = dataset_to_numpy(augmented_ds)

# Convert test data
X_test, y_test = dataset_to_numpy(test_ds)

print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)


13640it [00:36, 371.25it/s]
682it [00:01, 392.06it/s]

Train set: (13640, 3072) (13640,)
Test set: (682, 3072) (682,)


Training SVM model

In [11]:

model = SVC()
# Train the SVM model on the training data
model.fit(X_train, y_train)
# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)


Accuracy rate of SVM model

In [12]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.3680351906158358


Classification report of SVM model

In [13]:
# Retrieve the ordered list of class labels inferred from the test directory
# (subfolder names sorted alphabetically by image_dataset_from_directory).
class_names = test_ds.class_names

# Print a detailed per-class precision/recall/F1 report.
# y_test: true class indices (shape [N]); y_pred: predicted class indices (shape [N]).
# target_names ensures readable labels instead of numeric IDs.
print(classification_report(y_test, y_pred, target_names=class_names))


              precision    recall  f1-score   support

           0       0.23      0.27      0.25        11
           1       0.30      0.64      0.41        11
           2       0.50      0.36      0.42        11
           3       0.25      0.09      0.13        11
           4       0.21      0.27      0.24        11
           5       0.40      0.55      0.46        11
           6       0.14      0.09      0.11        11
           7       0.29      0.36      0.32        11
           8       0.22      0.45      0.29        11
           9       0.50      0.36      0.42        11
      A_caps       0.80      0.73      0.76        11
      B_caps       0.28      0.45      0.34        11
      C_caps       0.75      0.55      0.63        11
      D_caps       0.43      0.27      0.33        11
      E_caps       0.75      0.27      0.40        11
      F_caps       0.50      0.45      0.48        11
      G_caps       0.29      0.36      0.32        11
      H_caps       0.50    

SVM model tuning

In [14]:
# Base SVM classifier (defaults: RBF kernel, C=1.0, gamma='scale', probability=False).
model = SVC()

# Define the hyperparameter search space:
# - C: regularization strength (higher C → less regularization, tighter fit, risk of overfitting).
# - kernel: restrict to 'rbf' (nonlinear, usually strong for image features/HOG/PCA inputs).
# - class_weight: 'balanced' reweights classes inversely to their frequencies (robust to imbalance).
param_grid = [
    {
        'C': [5, 10, 15, 20],
        'kernel': ['rbf'],
        'class_weight': ['balanced']
    }
]

# GridSearchCV will:
# - perform cross-validation (default cv=5) over all parameter combinations,
# - optimize the default scorer (accuracy unless 'scoring' is specified),
# - refit the best model on the full training set.
grs = GridSearchCV(model, param_grid)

# Run the grid search on training data (X_train: features, y_train: labels).
grs.fit(X_train, y_train)

# Report the best hyperparameters found by cross-validation.
print("Best Hyper Parameters:", grs.best_params_)


Best Hyper Parameters: {'C': 15, 'class_weight': 'balanced', 'kernel': 'rbf'}


Best model's classification report

In [15]:
# Retrieve the refitted estimator with the best hyperparameters from GridSearchCV
best_model = grs.best_estimator_

# ---- Predict on the held-out test set ----
# X_test: feature matrix for test samples
y_pred = best_model.predict(X_test)

# Print precision/recall/F1 for each class + macro/weighted averages and overall accuracy.
print(classification_report(y_test, y_pred, target_names=class_names))



              precision    recall  f1-score   support

           0       0.19      0.27      0.22        11
           1       0.29      0.55      0.38        11
           2       0.43      0.55      0.48        11
           3       0.22      0.18      0.20        11
           4       0.40      0.18      0.25        11
           5       0.46      0.55      0.50        11
           6       0.30      0.27      0.29        11
           7       0.55      0.55      0.55        11
           8       0.27      0.64      0.38        11
           9       0.64      0.64      0.64        11
      A_caps       0.67      0.73      0.70        11
      B_caps       0.25      0.45      0.32        11
      C_caps       0.70      0.64      0.67        11
      D_caps       0.54      0.64      0.58        11
      E_caps       0.50      0.45      0.48        11
      F_caps       0.50      0.55      0.52        11
      G_caps       0.27      0.36      0.31        11
      H_caps       0.62    